In [ ]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

# Libraries and functions

In [1]:
import numpy as np
import pandas as pd
import pandasql as ps
from pandas import Series, DataFrame
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib as mpl
import sys
import time
import math
from tqdm import tqdm
from importlib import reload
from os.path import join, normpath
from bs4 import BeautifulSoup
import os

downloads = 'C:/Users/balob/Downloads'

#### To get the path

In [3]:
from pathlib import Path
path_dir = Path.cwd()
print(path_dir)
path_dir

C:\Users\balob\Documents\GITLAB\RB_BD


WindowsPath('C:/Users/balob/Documents/GITLAB/RB_BD')

In [2]:
pwd=!echo %cd%
pwd=str(pwd)
pwd=r"c:\Users\balob\Documents\GITLAB\NOC_Roamability_com"
pwd.replace('\\','/',100)

'c:/Users/balob/Documents/GITLAB/NOC_Roamability_com'

In [ ]:
%reset -f

# Examples

### Fetch data from MS SQL DB

In [ ]:
#Из базы данных OCS выгружаем TADIG

import pandas as pd
import pyodbc 
#Connect to OCSDBREP1 (BSS)
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.18.11.82;DATABASE=10028;UID=BSS;PWD=iKQVm40AZAmyRaw72LeY')

sql_srt=\
'''
SELECT DISTINCT mcc.mcc,c.country_name FROM RDB_NETWORK_IMSI_PREFIXES mcc
LEFT JOIN RDB_NETWORKS net
ON mcc.NETWORK_ID=net.NETWORK_ID
LEFT JOIN RDB_COUNTRIES c
ON c.country_id=net.country_id
'''

df_tadig = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
#df_tadig['TADIG_CODE_ID']=df_tadig['TADIG_CODE_ID'].astype('int')
#df_tadig['NETWORK_ID']=df_tadig['NETWORK_ID'].astype('int')
cnxn.close()

### Fetch data from Maria DB

In [ ]:
import mysql.connector #pip install mysql-connector-python-rf
import pandas as pd

cnxn = mysql.connector.connect(user='noc', password='WcQUzkXiXwoxnFfGnRxb',host='172.18.11.40',database='BSS')

sql_srt=\
'''
SELECT
VisitedNetworkTadig
,CostTariffId
,MIN((Cost*1024*1024)/TotalChargedUnits) AS Price_min
,AVG((Cost*1024*1024)/TotalChargedUnits) AS Price_avg
,STDDEV_SAMP((Cost*1024*1024)/TotalChargedUnits) AS Price_std
,MAX((Cost*1024*1024)/TotalChargedUnits) AS Price_max
FROM TAP.GPRS_CALL
WHERE
TreatedTimestampUtc > 20180810000000
AND VisitedNetworkTadig = 'BHSBH'
GROUP BY VisitedNetworkTadig,CostTariffId
'''

df_mariadb = pd.read_sql_query(sql_srt, cnxn)

cnxn.close()
df_mariadb.head()

### Soap API OCS Portal

In [184]:
#%reset -f
from zeep import Client
from zeep.wsse.username import UsernameToken

connect = 'prod' #'prod' or 'lab'

if connect == 'lab':
    user = 'lab_api@roamability.com'
    password = '25D55AD283AA400AF464C76D713C07AD' #Lab
    #api_link = 'https://172.20.39.7:8585/ocsapi/roamability/api/roamability.wsdl' #Lab
    api_link = 'https://labocsapi.roamability.com:6443/ocsapi/roamability/api/roamability.wsdl'
elif connect == 'prod':
    user = 'prod_api@roamability.com'
    password = '25D55AD283AA400AF464C76D713C07AD'
    api_link = 'http://172.20.35.12:8585/ocsapi/roamability/api/roamability.wsdl'

user_name_token = UsernameToken(user, password)
user_name_token.use_digest = True
client = Client(api_link, wsse=user_name_token)

reseller = client.service.getResellerInfo()
accounts = client.service.getAccounts()

if reseller.result.code == '1':
    print('You are connected to Reseller: {}.\
    \nAvailable credit for the Reseller: {:.0f} USD.\n'.\
    format(reseller.reseller.resellerName,reseller.reseller.availableCredit))
else:
    print('Error in getting Reseller info: {}'.format(reseller.result.description))

if accounts.result.code == '1':
    accounts_dict = {account.accountId:account.accountName for account in accounts.accounts.accounts}
    print('The Reseller contains the following accounts:')
    for i,[account_id,account_name] in enumerate(accounts_dict.items()):
        print(' {:<5}{} (accountID: {})'.format(i,account_name,account_id))
else:
    print('Error in getting Account info: {}'.format(accounts.result.description))
    
# Subscriber in lab in STI account
#subscriber_id = client.service.getSubscriberById('934311')
#subscriber_iccid = client.service.getSubscriberByICCID('8997219121000031446')
#subscriber_id

You are connected to Reseller: WMB Limited RT.    
Available credit for the Reseller: 291 USD.

The Reseller contains the following accounts:
 0    WM&B Limited Test SIM (accountID: 352496)
 1    WMB 5K Simcards order (accountID: 352537)
 2    Hoang Tran (accountID: 352541)
 3    Test1 (accountID: 352542)
 4    Test2 (accountID: 352543)


In [155]:
subscriber_imsi = client.service.getSubscriberByIMSI('425019613026594')
subscriber_imsi

{
    'result': {
        'code': '6',
        'description': 'Internal error',
        'errorUUID': 'cb1fa827-c1b7-4456-8388-888b6174f946'
    },
    'subscriber': None
}

### Testing DMI Steering API

In [ ]:
import requests
from pandas import DataFrame
from bs4 import BeautifulSoup
url = 'http://roamdb.roamability.com/SteeringAPI/Test'
payload = {'SponsorIMSI':'425019613990511',
           'RealIMSI':'425019613990511',
           'Mcc':'262',
           'SessionID':'1234567'}
response = requests.post(url,payload)
soup = BeautifulSoup(response.text, 'xml')
net_list = []
for net in soup.find_all('Network'):
    net_list.append([net.Rank.text,net.Mcc.text,net.Mnc.text,net.Name.text,net.Support4G.text,net.Discounted.text,net.Cost.text])
DataFrame(net_list,columns=['Rank','MCC','MNC','Operator_name','4G_Support','Discounted','Cost'])

### Send MSU

In [172]:
from bs4 import BeautifulSoup
import roamability as rb
reload(rb)

rb.test_func()
rb.print_imsi_prof('48790993070', '425019613000594', '79219600320', 'as_resp')

ogt = {'p4':'48790993070', 'partner':'97254120624', 'sure':'447797706411'}
#soup = BeautifulSoup(rb.sri4sm(ogt, msisdn),'xml')
#soup = BeautifulSoup(rb.prn(ogt, dgt, imsi),'xml')
#soup = BeautifulSoup(rb.sai(ogt, dgt, imsi),'xml')
#soup = BeautifulSoup(rb.sendSMS(ogt, imsi, msc, payload),'xml')
#soup

Greetings from Roamability!
<?xml version="1.0" encoding="utf-8"?>
<response>
<result>0</result><response>00077FF644027100003F0ABFFF0100000000000000FFFFFFF00001070052F0200000008A56780943229905000849521069310050490108290660109400504902084952106951000030</response>
<src_gt>79219600320</src_gt>
</response> 

This SIM card contains the following IMSIs:
Slot 0 425019613000594
Slot 1 260060149000594
Slot 2 425019615000003


 ### Format output

In [ ]:
import datetime as dt
print( 'start: {} {} :end'.format('one', 'two') ) #start: one two :end
print( 'start: {1} {0} :end'.format('one', 'two') ) #start: two one :end
print( 'start: test {:10} :end'.format('test') ) #start: test test       :end
print( 'start: test {:10} :end'.format(10) ) #start: test         10 :end
print( 'start: test {:>10} :end'.format('test') ) #start: test       test :end
print( 'start: test {:<10} :end'.format('test') ) #start: test test       :end
print( 'start: test {:_<10} :end'.format('test') ) #start: test test______ :end
print( 'start: test {:^10} :end'.format('test') ) #start: test    test    :end
print( 'start: test {:.10} :end'.format('test1test2test3') ) #start: test test1test2 :end
print( 'start: test {:_^10.5} :end'.format('test1test2test3') ) #start: test __test1___ :end
print( 'start: test {:d} :end'.format(42) ) #start: test 42 :end
print( 'start: test {:f} :end'.format(42) ) #start: test 42.000000 :end
print( 'start: test {:06.2f} :end'.format(3.141592) ) #start: test 003.14 :end
print( 'start: test {p[first]} {p[last]} :end'.format(p={'first': 'Jean-Luc', 'last': 'Picard'}) ) #start: test Jean-Luc Picard :end
print( 'start: test {p[4]} {p[5]} :end'.format(p=[4, 8, 15, 16, 23, 42]) ) #start: test 23 42 :end
print( 'start: test {:%Y-%m-%d %H:%M} :end'.format(dt.datetime(2001, 2, 3, 4, 5)) ) #start: test 2001-02-03 04:05 :end

### DataFrame examples

In [ ]:
#DataFrame
df=DataFrame(np.arange(200,212).reshape((3,4)) ,columns=list('abcd'),index=pd.date_range(start='1/1/2000 00:00:00',periods=3,freq='1H'))
df=DataFrame(np.arange(200,212).reshape((3,4)) ,columns=list('abcd'))
df=DataFrame(np.arange(200,212).reshape((3,4)) ,columns=list('abcd'),index=['Utah','Ohio','Texas'])

### Read csv example

In [ ]:
#Read CSV
df_tadig = pd.read_csv('TADIG.csv',sep=';',header=None,dtype={'MCC':object,'MNC':object},
                   names=['TADIG','MCC','MNC'],skiprows=0)

### Groupby examples

In [ ]:
#Вариант №1
#Группировка с применением различных типов агрегации к колонкам groupby().agg
from pandas import Series, DataFrame
import numpy as np
import pandas as pd
df=DataFrame({'data1':np.random.randint(0,10,size=20),'data2':np.random.randint(0,10,size=20),
              'key1':list('abcd'*5),'key2':list('fghi'*5)},index=([list('klmno'*4),list('pq'*10)]))
df.index.names=['ind1','ind2']
df.columns.names=['col_info']
df.groupby(['key1']).agg({'data1': np.mean, 'data2': np.size}) #Агрегация по одной колонке (вариант 1 задания группировки)
df.groupby(df['key1']).agg({'data1': np.mean, 'data2': np.size}) #Агрегация по одной колонке (вариант 2 задания группировки)
df.groupby(df['key1']).agg({'data1': [np.min,np.mean,np.max,np.size], 'data2': [np.min,np.mean,np.max,np.size]}) #Агрегация для каждого поля несколько функций
df.groupby(df['key1']).agg({'data1': [('f_min',np.min),('f_max',np.max)],'data2': [('f_min',np.min),('f_max',np.max)]}) #Агрегация для каждого поля несколько функций с переименованием
df.groupby(['key1','key2']).agg({'data1': np.mean, 'data2': np.size}) #Агрегация по нескольким колонкам
df.groupby(level=0).agg({'data1': np.mean, 'data2': np.size}) #Агрегация по одному индексу (вариант 1 указания индекса)
df.groupby(level='ind1').agg({'data1': np.mean, 'data2': np.size}) #Агрегация по одному индексу (вариант 2 указания индекса)
df.groupby(level=[0,1]).agg({'data1': np.mean, 'data2': np.size}) #Агрегация по нескольким индексам
df[['data1','data2']].groupby(level=[0,1]).agg(['min','max']) #Агрегация для каждого поля несколько функций
df[['data1','data2']].groupby(level=[0,1]).agg([('fmin','min'),('fmax','max')]) #Агрегация для каждого поля несколько функций с переименованием
()

In [ ]:
#Вариант №3
#Группировка .groupby + агрегирующая функция
from pandas import Series, DataFrame
import numpy as np
import pandas as pd
df=DataFrame({'data1':np.random.randint(0,10,size=20),'data2':np.random.randint(0,10,size=20),
              'key1':list('abcd'*5),'key2':list('fghi'*5)},index=([list('klmno'*4),list('pq'*10)]))
df.index.names=['ind1','ind2']
df.columns.names=['col_info']

# The Best Practice
df.groupby(['key1'],as_index=False)['data1'].max()
df.groupby(level=0)['data1'].max().reset_index()

df['data1'].groupby(df['key1']).mean() #Агрегация одного поля по одному ключу
df.groupby(df['key1']).mean() #Агрегация всех полей по одному ключу
df[['data1','data2']].groupby(df['key1']).mean() #Агрегация нескольких полей по одному ключу
df[['data1','data2']].groupby([df['key1'],df['key2']]).mean() #Агрегация нескольких полей по нескольким ключам
df[['data1','data2']].groupby(level=0).mean() #Агрегация по одному индексу (вариант 1 указания индекса)
df[['data1','data2']].groupby(level='ind1').mean() #Агрегация по одному индексу (вариант 2 указания индекса)
df[['data1','data2']].groupby(level=[0,1]).mean() #Агрегация по нескольким индексам
df[['data1','data2']].groupby(level=0,axis=1).mean() #Агрегация с группировкой по столбцам
#или
grouped=df['data1'].groupby(df['key1'])
grouped.mean()
#или
df.groupby(df['key1'])['data1'].mean()
#Чтобы добавить префикс к названию строки/столбца
#df['data1'].groupby(df['key1']).mean().add_prefix('mean_')
#df[['data1','data2']].groupby(level=0,axis=1).mean()
df['data1_mean']=df.groupby('key1')['data1'].transform(np.max)
()

### Search in Allowed Lists and others

In [97]:
downloads = 'C:/Users/balob/Downloads'

#Like expression for Allowed List
df_al=pd.read_csv(join(downloads,'DMI.dmi_allowed_list_export_Wed_Jan_16_2019.csv'))
#df_al_lab=pd.read_csv('LAB_DMI.dmi_allowed_list_export_Wed_Oct_03_2018.csv')

#df_dmi_diam_oper=pd.read_csv(join(download,''))
df_np=pd.read_csv(join(downloads,'DMI.dmi_netpfx_export_Sat_Dec_29_2018.csv'))
#df_did=pd.read_csv('HRR.hrr_did_export_Wed_Aug_08_2018.csv')
#f_tap_ocs=pd.read_csv('tap_ocs_analysis_180817.csv')

In [117]:
#df_al[(df_al['PLMN code'].str.startswith('MKDCC', na=False)) & (df_al['Sponsor']=='')]
df_al.loc[(df_al['PLMN code'].str.startswith('CHN', na=False)) &
          (df_al['Sponsor'].str.contains('P4_All', na=False)), ['Sponsor','PLMN code']].sort_values(by=['Sponsor','PLMN code'])

,Sponsor,PLMN code
2465,P4_All,CHNCT
895,P4_All Rated,CHNCT


In [39]:
str(df_np[df_np.Operator == ''].Prefix.unique().tolist())
df_np[(df_np.Operator.str.startswith('IND14',na=False))] #&(df_np.SSN==7)

,Operator,Prefix,Number/Range,SSN,MCC,MNC
1732,IND14,919815,1,7,404,10
1735,IND14,919815,1,149,404,10
1736,IND14,919816,1,149,404,10
1739,IND14,919816,1,7,404,10
2950,IND14,919829,1,7,404,10
2951,IND14,919829,1,149,404,10
2952,IND14,919831,1,149,404,10
2953,IND14,919831,1,7,404,10
2954,IND14,919840,1,7,404,10
2955,IND14,919840,1,149,404,10


In [53]:
str(df_np[(df_np.Prefix.str.startswith('6683',na=False))].Prefix.unique().tolist())
#df_np[(df_np.Prefix.str.startswith('6683',na=False))]

"['6683', '6683003201', '6683003202', '6683003203', '6683003240', '6683003231', '6683003232', '6683003233', '6683003200', '6683003230', '66830032000']"

In [ ]:
df_dmi_diam_oper[(df_dmi_diam_oper['DMI Operator'].str.startswith('', na=False))\
                 & (df_dmi_diam_oper['Realm'].str.contains('405', na=False))]

### Search in ALARMS by folder names

In [122]:
import os
path = 'c:/W_DATA_ROAM/ALARMS/'
#path = 'c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/PRICES/'
#path = c:\W_DATA_ROAM\ДОКУМЕНТАЦИЯ\NOC_HOW_IT_WORKS\
#path = 'c:/W_DATA_ROAM/ПРОЕКТЫ/'
#path = 'c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/PRICES/'
#path = 'c:/W_DATA_ROAM_DOC/ПРОЕКТЫ/'
names = os.listdir(path)
[path + name for name in names if name.upper().find('DEUD2')!=-1]

['c:/W_DATA_ROAM/ALARMS/MOC_Partner_DEUD2_#2031463_181114',
 'c:/W_DATA_ROAM/ALARMS/PC_DEUD2_1092860',
 'c:/W_DATA_ROAM/ALARMS/PS_DEUD2_P4',
 'c:/W_DATA_ROAM/ALARMS/PS_DEUD2_Partner_180508',
 'c:/W_DATA_ROAM/ALARMS/TAP_DEUD2_Missing_for_Weconnect_RT']

### Search in files by Tags

In [121]:
import os
from os.path import isfile, isdir, join, normpath
from tqdm import tqdm

paths = ['c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/NOC_HOW_IT_WORKS/',
        'c:/W_DATA_ROAM/МЕТОДИКИ/',
        'c:/W_DATA_ROAM/ПРОЕКТЫ/',
        'c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/NOC_KNOWN_ISSUES/',
        'c:/W_DATA_ROAM/МЕТОДИКИ/NOC_M_NEW_CONNECTION/',
        'c:/W_DATA_ROAM/ALARMS/',
        'c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/PRICES',]

# Если хотя бы один из тэгов присутствует. Нужно указать хотя бы один.
any_tags  = ['#lab_principle_of_operation']
# Если все теги присутствуют. Можно указать [] если нет обязательных тэгов.
all_tags = []

def find_txt_files_and_folders(path_name):
    file_names_all = os.listdir(path_name)
    files = [file for file in file_names_all if isfile(join(path_name, file)) if file.find('.txt')!=-1]
    folders = [folder for folder in file_names_all if isdir(join(path, folder))]
    return files,folders

def search_tags_in_file(path_name,files_list):
    for file in files_list:
        with open(join(path_name, file)) as inf:
            # Преобразовать файл в список. В качествер разделителя пробел. read читает весь файл.
            words_in_file = inf.read().lower().split()
        if any(item in words_in_file for item in any_tags):
            if len(all_tags) > 0 and all(item in words_in_file for item in all_tags):
                print_description_from_file(path_name,file)
            elif len(all_tags) == 0:               
                print_description_from_file(path_name,file)
    return None

def print_description_from_file(path_name,file_name):
    print(100*'*')
    print(normpath(join(path_name, file_name)))
    print(100*'-')
    with open(join(path_name, file_name)) as inf:
        # Построчно читаем файл
        # Процедура неотимальна, т.к. будет прочитан весь файл до конца
        for line in inf:
            line.strip()
            if '<DESCRIPTION>' in line:
                for line in inf:
                    line.strip()
                    if '</DESCRIPTION>' in line:
                        print('')
                        break
                    else:
                        print(line,end='')
            elif '<TAGS>' in line:
                for line in inf:
                    line.strip()
                    if '</TAGS>' in line:
                        print(30*'-')
                        break
                    else:
                        print(line,end='')
    return None

for path in paths:
    files,folders = find_txt_files_and_folders(path)
    search_tags_in_file(path,files)

    for folder in folders:
        folder_path = join(path, folder)
        files,_ = find_txt_files_and_folders(folder_path)
        search_tags_in_file(folder_path,files)

****************************************************************************************************
c:\W_DATA_ROAM\ДОКУМЕНТАЦИЯ\NOC_HOW_IT_WORKS\Redirect_data_traffic_to_lab_ocs.txt
----------------------------------------------------------------------------------------------------
#lab_principle_of_operation
------------------------------
Для перенаправления пакетного трафика в OCS лабы нужно использовать:
APN rb.partner.dev (не работает???)
APN rb.vepg.dev

****************************************************************************************************
c:\W_DATA_ROAM\ДОКУМЕНТАЦИЯ\NOC_HOW_IT_WORKS\Http_module_bypass\Http_module_bypass.txt
----------------------------------------------------------------------------------------------------
#ocs_principle_of_operation
#ocs_mop
#lab_principle_of_operation
#http_principle_of_operation
------------------------------
Пример МОР по перенаправлению IMSI в Http Module с OCS prod в OCS Lab.
Http Module отвечает за CS, соответственно, перена

### Hex to Dec conversion

In [125]:
# Hex to dec
print(int('1883d2a7',16))

# Dec to hex
print(hex(272252642))
print('%x' % 272252642)

# Dec to bin
print(bin(3))

# Bin to dec
print(int('11',2))

411292327
0x103a3ee2
103a3ee2
0b11
3


In [165]:
print(bin(1093))
a = '00010001000101'
# = '00000011001101'
print(int(a[-14:-11],2),'-',int(a[-11:-3],2),'-',int(a[-3:],2) )

0b10001000101
0 - 136 - 5


### Rename files

In [ ]:
import os
import re
path = 'c:/Users/balob/Downloads/new1/'
prefix = 'PYCON'
topic = ''
os.chdir(path)
names = sorted(filter(os.path.isfile, os.listdir('.')), key=os.path.getmtime)
for i,name in enumerate(names):
    short_name = re.sub('[!#?«»,() \+\-؟]','',name[:-4:][:50])
    if i < 10:
        number = '0'+str(i)
    else:
        number = str(i)
    new_name = '{}_{}_{}.mp4'.format(prefix,number,short_name)
    print('{} -> {}'.format(name,new_name))
    os.rename(name,new_name)

### Чтение логов с удаленного сервера SFTP (stat_GrayLog.log)

In [217]:
import paramiko
print_lines = 50
host = '172.19.11.191'
user = 'pavel'
secret = 'RzAZPjMjvujCPjU88bpevQp'
file_path_name = '/home/tracer/LOG/stat_GrayLog.log'
port = 22
client = paramiko.SSHClient()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
client.connect(hostname=host, username=user, password=secret, port=port)
sftp_client = client.open_sftp()
remote_file = sftp_client.open(file_path_name)
try:
    #for line in remote_file:
    for i,line in enumerate(reversed(list(remote_file))):
        if i < print_lines:
            if 'Total' in line:
                print(i,line,end='')
            #print(i,line,end='')
        else:
            break
finally:
    remote_file.close()
client.close()
# (https://stackoverflow.com/questions/1596963/read-a-file-from-server-with-ssh-using-python)

0 2018-12-14 05:00:45,537 - ss7_stat - INFO - rep_ss7_sqlite_db_py2. End. Total elapsed time 0:00:44.009712
8 2018-12-13 05:00:45,731 - ss7_stat - INFO - rep_ss7_sqlite_db_py2. End. Total elapsed time 0:00:43.732656
16 2018-12-12 05:00:44,040 - ss7_stat - INFO - rep_ss7_sqlite_db_py2. End. Total elapsed time 0:00:42.703231
24 2018-12-11 05:00:41,949 - ss7_stat - INFO - rep_ss7_sqlite_db_py2. End. Total elapsed time 0:00:39.759998
32 2018-12-10 05:00:42,655 - ss7_stat - INFO - rep_ss7_sqlite_db_py2. End. Total elapsed time 0:00:40.982727
40 2018-12-09 05:00:46,048 - ss7_stat - INFO - rep_ss7_sqlite_db_py2. End. Total elapsed time 0:00:42.357029
48 2018-12-08 05:00:44,158 - ss7_stat - INFO - rep_ss7_sqlite_db_py2. End. Total elapsed time 0:00:42.308123


### Pricing counting

In [192]:
moc=1.5
mtc=0
text=0.25
data=7.37

tariff=[moc,mtc,text,data]

print("Price P4")
for k,v in {'1.2':1.2,'1.2*1.5':1.2*1.5,'1.2*1.5*1.15':1.2*1.5*1.15,'1.2*1.5*1.30':1.2*1.5*1.30,'2':1.2*2}.items():
    print("Price {:<15}:".format(k),[round(i*v,6) for i in tariff])
print("\nPrice Partner")
for k,v in {'1.5':1.5,'1.5*1.15':1.5*1.15,'1.5*1.30':1.5*1.30,'2':2}.items():
    print("Price {:<15}:".format(k),[round(i*v,6) for i in tariff])

Price P4
Price 1.2            : [1.8, 0.0, 0.3, 8.844]
Price 1.2*1.5        : [2.7, 0.0, 0.45, 13.266]
Price 1.2*1.5*1.15   : [3.105, 0.0, 0.5175, 15.2559]
Price 1.2*1.5*1.30   : [3.51, 0.0, 0.585, 17.2458]
Price 2              : [3.6, 0.0, 0.6, 17.688]

Price Partner
Price 1.5            : [2.25, 0.0, 0.375, 11.055]
Price 1.5*1.15       : [2.5875, 0.0, 0.43125, 12.71325]
Price 1.5*1.30       : [2.925, 0.0, 0.4875, 14.3715]
Price 2              : [3.0, 0, 0.5, 14.74]


## Testing

In [ ]:
import mglearn
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
import numpy as np
import pandas as pd
from pandas import DataFrame,Series

In [ ]:
import requests
from bs4 import BeautifulSoup
resp = requests.get('http://www.cbr.ru/scripts/XML_daily.asp')
soup = BeautifulSoup(resp.content,'xml')
eur = soup.find(ID='R01239').Value.string
print('Курс EUR на составляет: {} RUB'.format(eur))

In [62]:
(lambda x, y: x + y)(5, 5)

10

In [67]:
import pandas as pd
from pandas import DataFrame
df=DataFrame(np.arange(200,212).reshape((3,4)) ,columns=list('abcd'),index=['Utah','Ohio','Texas'])
df['sum_val'] = df.apply(lambda x: x.a + x.b + x.c + x.d, axis=1)
display(df)
df.apply(lambda x: x.Utah, axis=0)

,a,b,c,d,sum_val
Utah,200,201,202,203,806
Ohio,204,205,206,207,822
Texas,208,209,210,211,838


a          200
b          201
c          202
d          203
sum_val    806
dtype: int64

In [73]:
from datetime import datetime
start_ts = int("1541023200")
end_ts = int("1547503200")

# if you encounter a "year is out of range" error the timestamp
# may be in milliseconds, try `ts /= 1000` in that case
print(datetime.utcfromtimestamp(start_ts).strftime('%Y-%m-%d %H:%M:%S'))
print(datetime.utcfromtimestamp(end_ts).strftime('%Y-%m-%d %H:%M:%S'))

2018-10-31 22:00:00
2019-01-14 22:00:00


In [64]:
import datetime
datetime.datetime(2018,11,30,4,0).timestamp()
#1541012400

1543539600.0

In [78]:
s1 = Series(pd.date_range(start='09/01/2013 14:00:00',periods=10,freq='1H'))
s1.values.astype("int64").reshape(-1, 1) // 10**9

array([[1378044000],
       [1378047600],
       [1378051200],
       [1378054800],
       [1378058400],
       [1378062000],
       [1378065600],
       [1378069200],
       [1378072800],
       [1378076400]], dtype=int64)

In [96]:
import calendar, time, datetime
import pandas as pd
from pandas import DataFrame
# 1378044000 == 09/01/2013 @ 2:00pm (UTC)
# Convert from human readable date to epoch without zone convertion
print(calendar.timegm(time.strptime('2013-09-01 14:00:00', '%Y-%m-%d %H:%M:%S')))
print(pd.Timestamp('09/01/2013 14:00',tz='UTC').value // 10**9)
# or for Series
s1 = Series(pd.date_range(start='09/01/2013 14:00:00',periods=3,freq='1d'))
print(s1.values.astype("int64").reshape(-1, 1) // 10**9)
# Convert from epoch to human readable date converted to local timestamp
print(time.strftime("%a, %d %b %Y %H:%M:%S", time.localtime(1378044000)))
print(datetime.datetime.fromtimestamp(1378044000)) # или с форматированием .strftime('%Y-%m-%d %H:%M:%S')
# Convert from epoch to human readable date in utc without convertion
print(datetime.datetime.utcfromtimestamp(1378044000))
# or for Series
df=DataFrame({'timestamp':[1378044000,1378130400,1378216800],'value':[25002,50590,50590]})
df['date']=pd.to_datetime(df['timestamp'], unit='s')
display(df)
# https://www.epochconverter.com/

1378044000
1378044000
[[1378044000]
 [1378130400]
 [1378216800]]
Sun, 01 Sep 2013 17:00:00
2013-09-01 17:00:00
2013-09-01 14:00:00


,timestamp,value,date
0,1378044000,25002,2013-09-01 14:00:00
1,1378130400,50590,2013-09-02 14:00:00
2,1378216800,50590,2013-09-03 14:00:00


In [103]:
# Загрузить данные из файлов
#----------------------------------

import pandas as pd
from pandas import DataFrame

downloads = 'C:/Users/balob/Downloads/DATA/'
result_file = 'result_file.csv'
files = ['rb_42501_25_06_20181.txt', 'rb_42501_2_09_20181.txt', 'rb_26006_25_06_20181.txt', 'rb_26006_2_09_20181.txt']

df_imsi = DataFrame(columns = ['IMSI', 'KI', 'OPC'])

for file in files:
    df_imsi = df_imsi.append(pd.read_table(join(downloads, file), sep=' ',header=0, engine='python'))

df_imsi.IMSI = df_imsi.IMSI.astype('int64')
df_imsi.sort_values(by = 'IMSI', inplace = True)

df_imsi.head()

,IMSI,KI,OPC
0,260060140890000,1AEA5D122A1F2476EBD9350F10C0EFDD,5EE04A60E740AF9890C397A899109B25
1,260060140890001,09B595FF54EEFE9E547237B4D7499202,9274F199697E5775100367CAE65B24D0
2,260060140890002,744A3D38AFBCBB0292FF9807D21E8B4D,8D7B5100C7BF361382490BFA632F8A55
3,260060140890003,2244EDED489D62B569EC8179FC73489F,2D63E23033BB91337C8DD03362984B44
4,260060140890004,FCB72503EA93EEB2A36CFBC23256697D,7697EA5B130950C519EDA1487C50CE2F


In [104]:
# Предварительная проверка диапазонов (опционально)
#----------------------------------

list_agg = [('IMSI_MIN','min'),('IMSI_MAX','max'),('NUM','count')]

df = df_imsi.groupby(df_imsi.IMSI.astype('str').str.slice(0,12))['IMSI'].agg(list_agg).reset_index()
df.IMSI = df.IMSI.astype('int64')
df.rename(columns = {'IMSI':'IMSI_RANGE'}, inplace = True)

df['imsi_shifted'] = df['IMSI_RANGE'].shift(-1)
df.fillna(0,inplace=True)
df['imsi_shifted'] = df['imsi_shifted'].astype('int64')

def check_imsi(x):
    x['imsi_num'] = x['IMSI_MAX'] - x['IMSI_MIN'] + 1
    x['check_range'] = x['imsi_shifted'] - x['IMSI_RANGE']
    if x['NUM'] == (x['IMSI_MAX'] - x['IMSI_MIN'] + 1):
        x['imsi_num_check'] = 'OK'
    else:
        x['imsi_num_check'] = 'NOK'
    return x

df = df.apply(check_imsi, axis=1)
df

,IMSI_RANGE,IMSI_MIN,IMSI_MAX,NUM,imsi_shifted,imsi_num,check_range,imsi_num_check
0,260060140890,260060140890000,260060140890999,1000,260060140891,1000,1,OK
1,260060140891,260060140891000,260060140891999,1000,260060140892,1000,1,OK
2,260060140892,260060140892000,260060140892999,1000,260060140893,1000,1,OK
3,260060140893,260060140893000,260060140893999,1000,260060140894,1000,1,OK
4,260060140894,260060140894000,260060140894999,1000,260060140895,1000,1,OK
5,260060140895,260060140895000,260060140895999,1000,260060140896,1000,1,OK
6,260060140896,260060140896000,260060140896999,1000,260060140897,1000,1,OK
7,260060140897,260060140897000,260060140897999,1000,260060140898,1000,1,OK
8,260060140898,260060140898000,260060140898999,1000,260060140899,1000,1,OK
9,260060140899,260060140899000,260060140899999,1000,260060149991,1000,9092,OK


In [109]:
# Из базы данных OCS выгружаем IMSI
#----------------------------------

import pandas as pd
import pyodbc 
#Connect to OCSDBREP1 (BSS)
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.18.11.82;DATABASE=10028;UID=BSS;PWD=iKQVm40AZAmyRaw72LeY')

sql_srt=\
'''
SELECT DISTINCT
r.RESELLER_NAME
,a.ACCOUNT_NAME
,ss.STATUS AS SUB_STATUS
,si.IMSI
FROM RESELLERS r
INNER JOIN ACCOUNTS a
ON r.RESELLER_ID = a.RESELLER_ID
INNER JOIN SUBSCRIBERS sr
ON sr.ACCOUNT_ID = a.ACCOUNT_ID
INNER JOIN SUBSCRIBER_STATUS ss
ON ss.SUBSCRIBER_ID = sr.SUBSCRIBER_ID
INNER JOIN SUBSCRIBER_IMSIS si
ON ss.SUBSCRIBER_ID = si.SUBSCRIBER_ID
WHERE
r.deleted is NULL
AND ss.END_DATE IS NULL
AND ss.status = 'Active'
AND
(si.IMSI BETWEEN '260060140890000' AND '260060140899999'
OR si.IMSI BETWEEN  '260060149991100' AND '260060149999999'
OR si.IMSI BETWEEN  '425019613998060' AND '425019613999999'
OR si.IMSI BETWEEN  '425019629920000' AND '425019629939999')
'''

df_ocs_imsi = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
df_ocs_imsi['IMSI'] = df_ocs_imsi['IMSI'].astype('int64')
cnxn.close()
df_ocs_imsi.head()


,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI
0,Acloud RT,Acloud Test,Active,260060149991109
1,Acloud RT,Acloud Test,Active,425019613998068
2,Acloud RT,Acloud Test,Active,425019613998069
3,Dialoq - Genesisclub RT,Genesisclub P4 Soft test,Active,260060149991110
4,Dialoq - Genesisclub RT,Genesisclub P4 Soft test,Active,260060149991115


In [110]:
# Объединяем IMSI с ключами с данными OCS
#----------------------------------

df_ocs_imsi_ki = pd.merge(df_imsi, df_ocs_imsi, how='left', on='IMSI')
list_agg = [('IMSI_MIN','min'),('IMSI_MAX','max'),('NUM','count')]
list_col = ['RESELLER_NAME', 'ACCOUNT_NAME', 'SUB_STATUS', df_ocs_imsi_ki.IMSI.astype('str').str.slice(0,12)]
df_ocs_imsi_ki_group = df_ocs_imsi_ki.groupby(list_col)['IMSI'].agg(list_agg).reset_index().sort_values(by='IMSI')

df_ocs_imsi_ki_group.head()
#df_ocs_imsi_ki_group[df_ocs_imsi_ki_group['RESELLER_NAME'] == 'Tottolli-Tech-Mobil']

,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
9,MCN Telecom,MCN Telecom Soft SIM,Active,260060140890,260060140890000,260060140890999,1000
10,MCN Telecom,MCN Telecom Soft SIM,Active,260060140891,260060140891000,260060140891999,1000
11,MCN Telecom,MCN Telecom Soft SIM,Active,260060140892,260060140892000,260060140892999,1000
12,MCN Telecom,MCN Telecom Soft SIM,Active,260060140893,260060140893000,260060140893999,1000
13,MCN Telecom,MCN Telecom Soft SIM,Active,260060140894,260060140894000,260060140894999,1000


In [113]:
df_ocs_imsi_ki_group[df_ocs_imsi_ki_group.IMSI == '425019613998']

,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
61,XOtel,XOtel Test account,Active,425019613998,425019613998070,425019613998070,1
51,Tapmobile RT,Tapmobile Test Sim cards,Active,425019613998,425019613998078,425019613998078,1
15,MCN Telecom,MCN Telecom Soft SIM,Active,425019613998,425019613998065,425019613998067,3
8,Joy telecom,Joy telecom Partner Soft test,Active,425019613998,425019613998088,425019613998089,2
33,Roamability Invertory,Partner Soft simcards inventory,Active,425019613998,425019613998091,425019613998996,906
49,Taisys,Test Sim cards,Active,425019613998,425019613998997,425019613998999,3
1,Acloud RT,Acloud Test,Active,425019613998,425019613998068,425019613998069,2
58,Watchdata SG,Watchdata Partner Soft test,Active,425019613998,425019613998090,425019613998090,1
3,Dialoq - Genesisclub RT,Genesisclub Partner Soft test,Active,425019613998,425019613998072,425019613998087,12
5,Flexiroam Limited -- TESTING at 2018-07,Flexiroam Test Soft simcard,Active,425019613998,425019613998060,425019613998064,5


In [107]:
df_ocs_imsi_ki_group

,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
9,MCN Telecom,MCN Telecom Soft SIM,Active,260060140890,260060140890000,260060140890999,1000
10,MCN Telecom,MCN Telecom Soft SIM,Active,260060140891,260060140891000,260060140891999,1000
11,MCN Telecom,MCN Telecom Soft SIM,Active,260060140892,260060140892000,260060140892999,1000
12,MCN Telecom,MCN Telecom Soft SIM,Active,260060140893,260060140893000,260060140893999,1000
13,MCN Telecom,MCN Telecom Soft SIM,Active,260060140894,260060140894000,260060140894999,1000
21,Roamability Invertory,P4 Soft simcards inventory,Active,260060140895,260060140895000,260060140895999,1000
22,Roamability Invertory,P4 Soft simcards inventory,Active,260060140896,260060140896000,260060140896999,1000
23,Roamability Invertory,P4 Soft simcards inventory,Active,260060140897,260060140897000,260060140897999,1000
54,Tottolli-Tech-Mobil,Tottolli S2 Soft,Active,260060140898,260060140898000,260060140898999,1000
55,Tottolli-Tech-Mobil,Tottolli S2 Soft,Active,260060140899,260060140899000,260060140899999,1000


In [126]:
%reset -f